# Methods to detect lined/squared paper

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

PARENT_DIR = os.path.dirname(os.path.dirname(os.path.realpath("FILEPATH")))
image_name = "004.jpg"
image = cv2.imread(os.path.join(PARENT_DIR, "images", image_name), cv2.IMREAD_COLOR)

# Check if image is loaded fine
if image is None:
    print('Error opening image')

# SHADOW REMOVAL
img = np.copy(image)
rgb_planes = cv2.split(img)

result_planes = []
result_norm_planes = []
for plane in rgb_planes:
    dilated_img = cv2.dilate(plane, np.ones((7, 7), np.uint8))
    bg_img = cv2.medianBlur(dilated_img, 21)
    diff_img = 255 - cv2.absdiff(plane, bg_img)
    norm_img = cv2.normalize(diff_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    result_planes.append(diff_img)
    result_norm_planes.append(norm_img)

result = cv2.merge(result_planes)
result_norm = cv2.merge(result_norm_planes)

# Show source image
plt.imshow(result_norm, cmap="gray")

## Hough Transform

In [ ]:
PROBABILISTIC = True

# gray = cv2.cvtColor(horizontal, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(result_norm, 50, 150, apertureSize=3)
lines_on_image = result_norm.copy()

if PROBABILISTIC:
    lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=100,maxLineGap=10)
    if lines is not None:
        print("Lines detected")
        # check if the lines are a considerable number (e.g. 5)
        if len(lines) > 5:
            print(len(lines))
            # check if the lines are long enough (e.g. one fourth of the image width)
            long_enough_lines = 0
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(lines_on_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
                if abs(x2 - x1) > image.shape[1] / 5:
                    long_enough_lines += 1
            print("Long enough lines:",long_enough_lines)
            if long_enough_lines > 5:
                print("The text is on lined paper")
    else:
        print("No lines detected")
else:
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)
    if lines is not None:
        print("Lines detected")    
        # check if the lines are a considerable number (e.g. 5)
        if len(lines) > 5:
            print(len(lines))
            # check if the lines are long enough (e.g. one fourth of the image width)
            long_enough_lines = 0
            for rho, theta in lines[:, 0]:
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho
                x1 = int(x0 + 1000 * (-b))
                y1 = int(y0 + 1000 * (a))
                x2 = int(x0 - 1000 * (-b))
                y2 = int(y0 - 1000 * (a))
                cv2.line(lines_on_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
                if abs(x2 - x1) > image.shape[1] / 4:
                    long_enough_lines += 1
            print("Long enough lines:", long_enough_lines)
            if long_enough_lines > 5:
                print("The text is on lined paper")
    else:
        print("No lines detected")

# show the image with lines
plt.imshow(lines_on_image, cmap="gray")

## Fourier Transform

In [ ]:
gray = cv2.cvtColor(result_norm, cv2.COLOR_BGR2GRAY)
dft = cv2.dft(np.float32(gray), flags=cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)
magnitude_spectrum = 20 * np.log(cv2.magnitude(dft_shift[:, :, 0], dft_shift[:, :, 1]))

# show the original image and the Fourier Transform of the image
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax[0].set_title("Input Image")
ax[0].axis("off")
ax[1].imshow(magnitude_spectrum, cmap="gray")
ax[1].set_title("Magnitude Spectrum")
ax[1].axis("off")
plt.show()
plt.close()
print(img.shape)
print(magnitude_spectrum.shape)

# If the magnitude spectrum has a peak at the middle of the image, for all the rows, then the image is on lined paper

# check if the magnitude spectrum has a peak at the middle of the image, for all the rows
middle = magnitude_spectrum.shape[1] // 2
peaks = []
for row in magnitude_spectrum:
    peaks.append(row[middle])
peaks = np.array(peaks)
# print(peaks)
plt.plot(peaks)
plt.show()
plt.close()
# check if the peaks are almost all above a certain threshold (e.g. the mean value of the spectrum)
# calculate the mean value of the spectrum
threshold = np.mean(magnitude_spectrum)
print(threshold)
print("Threshold:", threshold)

# check if the peaks are almost all above the threshold
# i.e. if 90% of the peaks are above the threshold
above_threshold = peaks > threshold
# print(above_threshold)
# print("Peaks above threshold:", np.sum(above_threshold))
if np.sum(above_threshold) > 0.95 * len(above_threshold):
    print("The text is on lined paper")
else:
    print("The text is not on lined paper")

# We reshape the probability from 0.8 to 1 to a percentage from 0 to 100
if np.sum(above_threshold) > 0.8 * len(above_threshold):
    print("\033[92m\nProbability of lined paper: \033[0m", ((np.sum(above_threshold)/len(above_threshold))-0.8)*500)

## Template Matching

In [ ]:
# TODO